Импорт библиотек и загрузка данных

In [ ]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords   
from nltk.stem import WordNetLemmatizer, SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics

In [ ]:
train_data = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", 
                    header=0,
                    delimiter="\t",
                    quoting=3)
test_data = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip',
                   header=0,
                   delimiter="\t",
                   quoting=3 )

In [ ]:
train_data

# Обработка данных

Удаление HTML-тегов, токенизация и леммизация, удаление мусорных слов. 

In [ ]:
stopwords = stopwords.words('english')
stemmer = SnowballStemmer("english")
def my_tokenizer(sample):
    words = nltk.word_tokenize(sample)
    tokens = [word for word in words if word.isalnum()]
    tokens = [word for word in tokens if not word.isdigit()]
    meaningful_words = [stemmer.stem(t) for t in tokens if t not in stopwords]
    word_list = [lemmatizer.lemmatize(w) for w in meaningful_words]
    
    return word_list

def my_preprocessor(sample):
    no_tags_text = BeautifulSoup(sample).get_text()  
    review_text = no_tags_text.lower()
    
    return review_text

Сконвертируем полученные данные в матрицу признаков.

In [ ]:
lemmatizer = WordNetLemmatizer()
vectorizer = CountVectorizer(analyzer = "word", tokenizer = my_tokenizer, preprocessor = my_preprocessor,
                             stop_words = None, max_features = 10000) 

In [ ]:
train_data_features = vectorizer.fit_transform(train_data.review)
train_data_features = train_data_features.toarray()

test_data_features = vectorizer.transform(test_data.review)
test_data_features = test_data_features.toarray()

Проверим полученные данные.

In [ ]:
train_data_features.shape

# Машинное обучение

Часть размеченных данных была разделена для поиска оптимальных параметров модели с помощью GridSearchCV. Для экономии времени на Kaggle поиск не проводится.

In [ ]:
roc_auc_scorer = metrics.make_scorer(metrics.roc_auc_score)

In [ ]:
indices = np.random.choice(train_data_features.shape[0], size=3000, replace=False)
X = train_data_features[indices]
y = train_data.sentiment
y = y[indices]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
#params = {'n_estimators':[250, 300, 350, 400, 500, 600], 'criterion':['gini', 'entropy'], 'max_depth':[None, 20, 30, 40, 50]}
#rf = RandomForestClassifier(random_state = 0)
#clf = GridSearchCV(rf, param_grid = params, scoring = roc_auc_scorer, cv = 5, n_jobs = 6)
#clf.fit(X_train, y_train)
#print('Best score: {}'.format(clf.best_score_))
#print('Best parameters: {}'.format(clf.best_params_))

#Полученные результаты
#Best score: 0.8468461689758847
#Best parameters: {'criterion': 'gini', 'max_depth': None, 'n_estimators': 600}


Классификатор с лучшими параметрами в последствии был обучен на всех размечнных данных.

In [ ]:
#best_forest = clf.best_estimator_
best_forest = RandomForestClassifier(criterion = 'gini', max_depth = None, n_estimators = 600)
best_forest.fit(train_data_features, train_data.sentiment)

Сделано предсказание, результаты выгружены в файл.

In [ ]:
result = best_forest.predict(test_data_features)

In [ ]:
output = pd.DataFrame( data={"id":test_data.id, "sentiment":result} )

output.to_csv( "./submission.csv", index=False, quoting=3 )